In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('train.csv')
train_Y= train['open_flag']
train_X =train.drop(['row_id','user_id','grass_date','open_flag'],axis=1)
train_X['last_open_day']=train_X['last_open_day'].replace(to_replace = 'Never open',value=-999).astype('int64')
train_X['last_login_day']=train_X['last_login_day'].replace(to_replace = 'Never login',value=-999).astype('int64')
train_X['last_checkout_day']=train_X['last_checkout_day'].replace(to_replace = 'Never checkout',value=-999).astype('int64')
train_X.head()

In [ ]:
users = pd.read_csv('users.csv')
users.head()

In [ ]:
test = pd.read_csv('test.csv')
test_X =test.drop(['row_id','user_id','grass_date'],axis=1)
test_X['last_open_day']=test_X['last_open_day'].replace(to_replace = 'Never open',value=-999).astype('int64')
test_X['last_login_day']=test_X['last_login_day'].replace(to_replace = 'Never login',value=-999).astype('int64')
test_X['last_checkout_day']=test_X['last_checkout_day'].replace(to_replace = 'Never checkout',value=-999).astype('int64')
test_X.head()

In [ ]:
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import math
train_x,val_x,train_y,val_y=train_test_split(train_X,train_Y,test_size=0.3)
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

In [ ]:
#權重改變
weight_ratio = float(len(train_y[train_y == 1]))/float(len(train_y[train_y == 0]))
w_array = np.array([1]*train_y.shape[0])
w_array[train_y==1] = 1/weight_ratio
w_array[train_y==0] = 1/(1- weight_ratio)
w_array

In [ ]:
#客製化:計算Matthews correlation coefficient
dval=xgb.DMatrix(val_x,val_y)
def calculate_Matthews_correlation(preds, dtrain):
    labels = dtrain.get_label()#TRUE
    #print(preds)#大於0.5為1的預測
    #print(labels)
    preds = np.array(preds)
    preds[preds>0.5]=1
    preds[preds<0.5]=0
    c_matrix = sklearn.metrics.confusion_matrix(labels, preds)
    TN = c_matrix[0,0]
    FN = c_matrix[1,0]
    FP = c_matrix[0,1]
    TP = c_matrix[1,1]
    Matthews_cor = ( (TP*TN)-(FP*FN) ) /  math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return 'Matthews_cor', Matthews_cor

In [ ]:
#均勻表製作
def dt(data,ori_uniform,linsp,new_lisp):
    #new_data = 需要改變dataframe
    #ori_uniform = 原來均勻表格式
    #linsp = 需要改的名稱
    #new_lsip = 新的分割
    import copy
    new_data = copy.deepcopy(data)
    ##注意地點:此處len(ori_uniform[col])是要取得表中總共有幾個列(實驗)
    for col in linsp:
        for row in range(0,len(ori_uniform[col])):
            new_data.loc[row,col] = new_lisp[int(data.loc[row,col]-1)]
    return new_data
#函數版 因使用partial 所以必須把C_函數放進來
#此版應該可以再精簡，期待有緣人縮改
def ug(data):
    import copy
    from functools import partial
    new_data =copy.deepcopy(data) 
    def C_2(x,name):#二循環
        if (x%2)==1:
            x = eval(str(name+'[0]'))   
        elif (x%2)==0:
            x = eval(str(name+'[1]'))
        return x
    def C_3(x,name):#三循環
        if (x%3)==1:
            x = eval(str(name+'[0]'))
        elif (x%3)==2:
            x = eval(str(name+'[1]'))      
        elif (x%3)==0:
            x = eval(str(name+'[2]'))
        return x    
    def C_4(x,name):#四循環
        if (x%4)==1:
            x = eval(str(name+'[0]'))
        
        elif (x%4)==2:
            x = eval(str(name+'[1]'))
        
        elif (x%4)==3:
            x = eval(str(name+'[2]'))
        
        elif (x%4)==0:
            x = eval(str(name+'[3]'))
        return x
    def C_5(x,name):#5循環
        if (x%5)==1:
            x = eval(str(name+'[0]'))
        
        elif (x%5)==2:
            x = eval(str(name+'[1]'))
        
        elif (x%5)==3:
            x = eval(str(name+'[2]'))
        
        elif (x%5)==4:
            x = eval(str(name+'[3]'))
        
        elif (x%5)==0:
            x = eval(str(name+'[4]'))
        return x
    ##注意地點:此處eval(col)是要取得原參數設定col中有幾個元素，這邊使用廣域變數取得到的
    for ind,col in enumerate(new_data.columns):
        if len(eval(col)) ==2:
            C_2 = partial(C_2,name= new_data.columns[ind])
            new_data[col] = list(map(C_2,new_data[col]))
        elif len(eval(col)) ==3:
            C_3 = partial(C_3,name= new_data.columns[ind])
            new_data[col] = list(map(C_3,new_data[col]))
        elif len(eval(col)) ==4:
            C_4 = partial(C_4,name= new_data.columns[ind])
            new_data[col] = list(map(C_4,new_data[col]))
        elif len(eval(col)) ==5:
            C_5 = partial(C_5,name= new_data.columns[ind])
            new_data[col] = list(map(C_5,new_data[col]))      
    return (new_data,data)#newdata =新資料,data = 原始資料

In [ ]:
###超參數預設
learning_rate=[0.03,0.1,0.3]
subsample=[0.7,0.9,1]
colsample_bytree=[0.7,0.9,1]
max_depth=[3,6,9]
min_child_weight=[0,0.5,2]
gamma=[0,0.3,3]
reg_lambda=[0,0.5,3]
###讀取均勻表U30_13
def getuniform():
    U30_13 = pd.read_csv('U30_13.csv',header=None)
    U30_7 = [1,2,3,4,6,12,13]
    U30_7 = U30_13.iloc[:,np.array(U30_7)-1]
    return U30_7
U30_7 = getuniform()
###超參數配置
columns = ['learning_rate','subsample','colsample_bytree','max_depth','min_child_weight','gamma','reg_lambda']
U30_7.columns = columns
#print(U30_7)
(U30_7_param,U30_7)=ug(U30_7)


U30_7_param['learning_rate']=U30_7['learning_rate']#設回初始
U30_7_param['gamma']=U30_7['gamma']#設回初始
U30_7_param['reg_lambda']=U30_7['reg_lambda']#設回初始
U30_7_param['min_child_weight']=U30_7['min_child_weight']#設回初始
U30_7_param=dt(U30_7_param,U30_7,linsp=['learning_rate'],new_lisp =list(np.linspace(0.01,0.03,num= 30)))
U30_7_param=dt(U30_7_param,U30_7,linsp=['gamma'],new_lisp =list(np.linspace(0.01,5,num= 30)))
U30_7_param=dt(U30_7_param,U30_7,linsp=['reg_lambda'],new_lisp =list(np.linspace(0.01,3,num= 30)))
U30_7_param=dt(U30_7_param,U30_7,linsp=['min_child_weight'],new_lisp =list(np.linspace(0.01,3,num= 30)))

In [ ]:
#根據實驗計算fitness
for expnum in range(len(U30_7_param)):
    exp = U30_7_param.iloc[expnum,:].to_dict()
    param_dist = {'objective':'binary:logistic',
         'learning_rate':exp['learning_rate'],'gamma':exp['gamma'],
         'subsample':exp['subsample'],'colsample_bytree ':exp['colsample_bytree'],
         'reg_lambda ':exp['reg_lambda'],'max_depth ':exp['max_depth'],
         'min_child_weight ':exp['min_child_weight']}
    print(f'param_dist:{param_dist}')
    clf = xgb.XGBClassifier(n_estimators=1000,**param_dist)
    clf.fit(train_x, train_y,
    eval_set=[(train_x, train_y), (val_x, val_y)],
    eval_metric=['auc','error','logloss']#預設與自訂義無法放一起
    #eval_metric=calculate_Matthews_correlation
    ,early_stopping_rounds=100
    ,sample_weight=w_array
    ,verbose=True)
    evals_result = clf.evals_result()
    #取驗證集的Matthews_cor
    U30_7_param.loc[expnum,'fitness'] = evals_result['validation_1']['logloss'][-100]#根據early_stopping_rounds設置Matthews_cor

In [ ]:
#最佳超參數設置
expbest = np.where(U30_7_param['fitness']==np.min(U30_7_param['fitness']))[0][0]
exp = U30_7_param.iloc[expbest,:].to_dict()
param_dist = {'objective':'binary:logistic',
         'learning_rate':exp['learning_rate'],'gamma':exp['gamma'],
         'subsample':exp['subsample'],'colsample_bytree ':exp['colsample_bytree'],
         'reg_lambda ':exp['reg_lambda'],'max_depth ':exp['max_depth'],
         'min_child_weight ':exp['min_child_weight']}
print(f'param_dist:{param_dist}')
clf = xgb.XGBClassifier(n_estimators=1000,**param_dist)
clf.fit(train_x, train_y,
    eval_set=[(train_x, train_y), (val_x, val_y)],
    eval_metric=['logloss','auc','error',]#預設與自訂義無法放一起
    #eval_metric=calculate_Matthews_correlation
    ,early_stopping_rounds=100
    ,sample_weight=w_array
    ,verbose=True)
evals_result = clf.evals_result()

In [ ]:
# 画出损失函数的变化情况
def plot_logloss(model):
    results = model.evals_result_
    #print(results)
    epochs = len(results['validation_0']['logloss'])
    x_axis = range(0, epochs)
    print(f'epochs:{epochs}')
    # plot log loss
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    #print(results['validation_1']['logloss'])
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    plt.ylabel('Log Loss')
    plt.title('XGDboost Log Loss')
    plt.show()
    # plot classification error
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    plt.ylabel('Classification Error')
    plt.title('XGBoostClassification Error')
    plt.show()
    # plot auc
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['auc'], label='Train')
    ax.plot(x_axis, results['validation_1']['auc'], label='Test')
    ax.legend()
    plt.ylabel('auc')
    plt.title('XGBoostClassification auc')
    plt.show()
import matplotlib.pyplot as plt
plot_logloss(clf)

In [ ]:
###驗證函數val_x, val_y
val_y_predictions = clf.predict(val_x)
val_y_predprob = clf.predict_proba(val_x)
from sklearn import metrics
print ('AUC: %.4f' % metrics.roc_auc_score(val_y,val_y_predprob[:,1]))
print ('ACC: %.4f' % metrics.accuracy_score(val_y,val_y_predictions))
print ('Recall: %.4f' % metrics.recall_score(val_y,val_y_predictions))
print ('F1-score: %.4f' %metrics.f1_score(val_y,val_y_predictions))
print ('Matthews_corrcoef: %.4f' %metrics.matthews_corrcoef(val_y,val_y_predictions))
print ('Precesion: %.4f' %metrics.precision_score(val_y,val_y_predictions))
print(metrics.confusion_matrix(val_y,val_y_predictions))

In [ ]:
#Predict training set:
test_X_predictions = clf.predict(test_X)
test_X_predprob = clf.predict_proba(test_X)
def sub_df(clf,test_X,test):
    test_X_predictions = clf.predict(test_X)
    test = test.drop([col for col in test.columns if col!='row_id'],axis=1)
    test['open_flag'] = test_X_predictions
    test.to_csv('sub.csv',index=False)
sub_df(clf,test_X,test)

In [ ]:
model_path = "./model.mdl"
# 保存模型
clf.get_booster().save_model(model_path)